SwapAI is an AI-powered agent that helps users perform token swaps and transfers on the Base blockchain. It takes natural language requests like *“Swap 0.5 ETH to USDC with 1% slippage”* or *“Send 20 USDC to 0xabc…”*, extracts the parameters, and executes the right action using integrated tools. The agent can also handle general user questions beyond swaps and sends.


#### **The goal**

* Make blockchain interactions easier through conversational AI.
* Ensure clarity by confirming parameters before execution.
* Support structured outputs (markdown) for front-end rendering.
* Balance between tool usage (for swaps/sends) and general reasoning (for normal questions).

---

#### **Stack**

* **Python + Jupyter Notebook** (development & testing)
* **LangChain** (LLM orchestration, tools binding)
* **LangGraph** (state management, branching logic, tool handling)
* **Google Generative AI (Gemini)** or **OpenAI GPT** (LLM backbone)
* **dotenv** (manage API keys and environment variables)

---

#### **Why LangChain**

* Provides a clean interface to connect LLMs with tools.
* Offers message-based state management through LangGraph.
* Makes it easier to maintain context windows and structured workflows.
* Prebuilt utilities like `ToolNode` and `add_messages` reduce boilerplate.
* Extensible: easy to add new tools, memory, or different LLM backends.


---

#### Imports

1. ***from system_prompt import DEFAULT_SYSTEM_PROMPT***  
This imports the DEFAULT_SYSTEM_PROMPT from the system_prompt.py file in the repo.

2. ***from dotenv import load_dotenv***  
Loads environment variables from a .env file into your Python environment. This lets us keep API keys outside the code in a .env file with *GOOGLE_API_KEY=your API key* as its content

3. ***from typing import Annotated, Sequence, List, Optional***    
The typing module is used for type hints.  

    **Annotated** adds metadata to a type.  

    **Sequence** means any ordered collection.    

    **List** is specifically a python list.  

    **Optional** means the value can be None.   

4. ***from typing_extensions import TypedDict***  
    **TypedDict** lets you define dictionary “schemas” with fixed keys and types.

5. ***from langgraph.graph import StateGraph, END***  

    **StateGraph** is the main component for building a state machine or a flowchart for an AI application.

    **END** is a special, built-in node that signifies the end of a process.

    StateGraph is used to orchestrate complex, multi-step workflows for an AI agent, and END is how you tell the workflow to stop.

6. ***from langchain_core.messages import AIMessage, HumanMessage, SystemMessage***  
These are data structures that represent the different roles in a conversation. They provide the structured format that chat models require to understand the flow and context of a conversation.  

    **BaseMessage:** The base class for all chat messages in LangChain.  

    **Subclasses:** 

        HumanMessage: Represents input from the end-user. 

        AIMessage: Represents a response generated by the AI model.

        SystemMessage: Sets the overall behavior, persona, or high-level instructions for the AI. 

        ToolMessage: Stores a tool's response. Needed so the conversation history includes not just what the AI says, but also what tools returned.

7. ***from langchain_google_genai import ChatGoogleGenerativeAI***  
This is the LangChain wrapper for Google's Gemini family of models. It allows the code to commmunicate with the Gemini API in a standardised way that fits into the LangChain ecosystem.

8. ***from langchain_core.tools import tool***  
This turns a python function into a tool that the AI can use. 

9. ***from langgraph.graph.message import add_messages***  
    **add_messages** is a reducer function. This means: every time a node returns new messages, instead of replacing the old ones, it appends them to the state automatically.

10. ***from langgraph.prebuilt import ToolNode***  
    **ToolNode** is a prebuilt node that knows how to call tools. You pass it your tools list, and it manages execution + returning results.

In [20]:
from system_prompt import DEFAULT_SYSTEM_PROMPT

from dotenv import load_dotenv 

from typing import Annotated, Sequence, List, Optional
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

In [21]:
load_dotenv()   # This loads the .env file which contains the API key

True

#### Config Dictionary  
Defines system prompt, creativity (temperature), response length, and model.  

Keeps settings in one place → easier to tweak, cleaner code, and easier to switch models.

In [22]:
config = {
    "system_prompt": DEFAULT_SYSTEM_PROMPT,
    "temperature": 0.9,
    "max_output_tokens": 1008,
    "model": "gemini-2.5-flash-lite"
}

#### Class Creation

**AgentState**  

- Defines what data flows between nodes in your LangGraph (e.g., messages, tool results).

- Keeps the workflow consistent and prevents errors (e.g., wrong data types).

- Makes the agent easier to extend later (e.g., add memory, logs).  
    

**InstructionStep**

- Gives a strict schema for step-by-step instructions.

-  Ensures output is structured and predictable for your frontend.

- Makes parsing and rendering simple (no guesswork in how the AI formats steps).

In [23]:
class AgentState(TypedDict):
    """Represents the state of the agent's workflow."""
    # A list of messages, annotated with a reducer to append new messages
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [24]:
# Define a type for structured step-by-step instructions
class InstructionStep(TypedDict):
    # Short heading for the step
    title: str
    # List of detail lines (each step can have multiple sub-points)
    details: List[str]

#### Tool Definition

SwapAI uses three tools to handle different types of user intents:

##### `swap_tokens`

* **Purpose:** Handles token swaps on the Base network.
* **Example:** `Swap 0.5 ETH to USDC`.
* **Parameters:**

  * `amount` *(float)* → amount of tokens to swap.
  * `from_token` *(str)* → token being swapped from.
  * `to_token` *(str)* → token being swapped to.
  * `slippage` *(float, default: 1.0)* → allowed price movement during the swap.

---

##### `send_tokens`

* **Purpose:** Transfers tokens to another wallet address.
* **Example:** `Send 5 USDC to 0xabc...`.
* **Parameters:**

  * `amount` *(float)* → amount of tokens to send.
  * `token` *(str)* → token being transferred.
  * `recipient` *(str)* → wallet address of the recipient.

---

##### `give_instructions`

* **Purpose:** Provides step-by-step guidance in Markdown format.
* **Example:** `How do I connect my wallet?`.
* **Parameters:**

  * `topic` *(str)* → the subject the user needs guidance on.

---

This structure ensures SwapAI can both execute transactions and educate users depending on their request.



In [25]:
# Tools

@tool
def swap_tokens(amount: float, from_token: str, to_token: str, slippage: float = 1.0):
    """Produce a mock swap of tokens following the instructions in 'DEFAULT_SYSTEM_PROMPT'"""
    return {
        "status": "success",
        "tx_hash": "0xmockswap123",
        "details": f"Swapped {amount} {from_token} to {to_token} with {slippage}% slippage"
    }

# Mock tool to simulate sending tokens
@tool
def send_tokens(amount: float, token: str, recipient: str):
    """Produce a mock send transaction following the instructions in 'DEFAULT_SYSTEM_PROMPT '"""
    return {
        "status": "success",
        "tx_hash": "0xmocksend456",
        "details": f"Sent {amount} {token} to {recipient}"
    }

@tool
def give_instructions(
    title: str,
    summary: str,
    steps: List[InstructionStep],
    notes: Optional[List[str]] = None,
    warnings: Optional[List[str]] = None,
    checklist: Optional[List[str]] = None,
) -> str:
    """Generate step-by-step instructions in markdown format with optional notes, warnings, and checklist."""
    
    md = f"## {title}\n"
    md += f"**Short summary:** {summary}\n\n"
    md += "### Steps\n"
    for i, step in enumerate(steps, 1):
        md += f"{i}. **{step['title']}**\n"
        for d in step["details"]:
            md += f"   - {d}\n"
        md += "\n"

    if notes:
        for n in notes:
            md += f"> **Note:** {n}\n"
    if warnings:
        for w in warnings:
            md += f"> **Warning:** {w}\n"
    if checklist:
        md += "\n**Checklist**\n"
        for c in checklist:
            md += f"✅ {c}\n"

    return md

tools = [swap_tokens, send_tokens, give_instructions]


##### LLM Setup  
This block initializes the `ChatGoogleGenerativeAI` model using parameters stored in the `config` file:  

- **model** → selects which LLM to use.  
- **temperature** → controls randomness in responses.  
- **max_output_tokens** → limits the length of generated outputs.  
- **system_instruction** → provides a system prompt to guide behavior.  

Finally, `.bind_tools(tools)` connects the LLM to our custom tools (`swap`, `send`, `give_instruction`). This allows the model to not only generate text but also take actions by invoking the tools during execution.  


In [26]:
llm = ChatGoogleGenerativeAI(
    model=config["model"],
    temperature=config["temperature"],
    max_output_tokens=config["max_output_tokens"],
    model_kwargs={"system_instruction": config["system_prompt"]}
).bind_tools(tools)

#### Swap Agent with Sliding Context Window

The `swap_agent` function manages how the agent handles conversations and tool calls and also manages the conversation history for the agent while ensuring it never exceeds a fixed context size (MAX_CONTEXT).  


**Key Points**

System Prompt: Always prepends the fixed system instruction to maintain agent behavior.

Growing History: Adds each new response to the conversation.

Sliding Window: Once the number of messages exceeds MAX_CONTEXT, the oldest message is dropped, keeping the latest ones in memory.

Stable Memory Size: Ensures the model works with recent context only, avoiding overflow while still maintaining continuity.

LLM Invocation: Calls the `llm` with both the system prompt and the message history. The response can be normal text or a tool call.  

State Update: Returns a new `AgentState` dictionary. Appends the model’s response to the list of messages.  

In [27]:
MAX_CONTEXT = 16

def swap_agent(state: AgentState) -> AgentState:
    system_prompt = SystemMessage(content=DEFAULT_SYSTEM_PROMPT)

    # Copy current messages
    messages = list(state["messages"])

    # Call LLM with system prompt + current history
    response = llm.invoke([system_prompt] + messages)

    # Enforce MAX_CONTEXT by popping oldest if too long
    if len(messages) > MAX_CONTEXT:
        messages.pop(0)
    
    # Add the new response
    messages.append(response)

    return {"messages": messages}


#### should_continue – Control Flow for Tool Usage

This function decides whether the agent should continue the conversation flow (e.g., calling a tool) or stop.

**Key Points**

- Input: Takes the current AgentState, which holds all past messages.

- Last Message Check: Looks at the most recent message in the conversation.

- Decision Rule: 
    * If the last message does not request a tool call, the process ends ("end").

    * If the last message contains a tool call, the agent should continue ("continue").

**Purpose**

This acts as a `router` in the agent’s workflow, ensuring the agent only proceeds with tool execution when necessary, otherwise stopping cleanly.

In [28]:
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        return "end"
    else: 
        return "continue"

#### Building the Agent Graph

This section defines the workflow graph for the agent using `StateGraph`. The graph manages how the agent moves between reasoning steps (LLM) and tool execution.

**Key Steps**

- Initialize Graph: A new `StateGraph` is created with `AgentState` as its state schema.

- Add Nodes:

   * `swap_agent`: Represents the LLM node that handles responses and system prompt context.
   * `tools`: A `ToolNode` that wraps all available tools for the agent.

- Set Entry Point: The graph starts execution at `swap_agent`, ensuring the agent always reasons first before calling tools.

- Conditional Edges: From `swap_agent`, the agent decides what to do using the `should_continue` function:

    * "continue" → move to `tools` if a tool call is requested.
    * "end" → stop execution if the reply is final (no tool call).

- Regular Edge: After tools are executed, control always flows back to `swap_agent` so the LLM can process results.

- Compile Graph: Finally, the graph is compiled into `app`, making it ready for use as the full agent workflow.

**Purpose**

This setup allows the agent to loop between LLM reasoning and tool use as needed, while also cleanly ending conversations when tool calls aren’t required.


In [29]:
# Create a new graph with the defined state
graph = StateGraph(AgentState)

# Add nodes
graph.add_node("swap_agent", swap_agent)

tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

# Set entry point
graph.set_entry_point("swap_agent")

# Define edges

# The llm node can go to tool or end based on the condition
graph.add_conditional_edges(
    "swap_agent",
    should_continue,
    {
        "continue": "tools",   # if tool call, go to tools
        "end": END     # else just end (LLM reply is final)
    },
)


graph.add_edge("tools", "swap_agent")

# compile the graph
app = graph.compile()

#### Chat Interface Function

**Function**: `chat_with_agent`

This function provides a simple interface for interacting with the agent.

**How it Works**

- Initialize History: If no conversation history is provided, a new empty list is created.

- Add User Input: The user’s message is wrapped as a `HumanMessage` and appended to history.

- Prepare State: The current conversation history is packaged into a `state` dictionary for the agent.

- Invoke Agent: The compiled agent graph (`app`) is called with the state.

- Update History: Any new `AIMessage` responses from the agent are extracted and added to history.

- Return: The updated conversation history is returned, maintaining context across turns.

**Purpose**

This function acts as the chat loop, ensuring smooth back-and-forth conversation with memory of past messages.


In [30]:
def chat_with_agent(user_input: str, history=None):
    if history is None:
        history = []
    
    # The history list currently holds everything from PREVIOUS runs.
    
    # Store the user input and set the initial state
    history.append(HumanMessage(content=user_input))
    state = {"messages": history}

    # Keep track of the length *before* the run starts
    initial_length = len(history)

    result = app.invoke(state)

    # The new messages usually start with the LLM's response (AIMessage)
    # and might include a ToolMessage or a second AIMessage.
    new_messages = result["messages"][initial_length:]

    # reassign `history` to the full message list from the result.
    history = result["messages"] # Reassign history to the full, updated list
    
    # Return the newly generated messages, which is what you want to output
    return history, new_messages

#### Interactive Chat Loop

This code creates a continuous chat session between the user and the AI agent. It allows for multi-turn conversations that maintain context until the user decides to exit.

**How It Works**

- Initialize History: `history = []` starts an empty list to store conversation messages.

- Start Chat Loop: The `while True` loop keeps the chat running indefinitely.

- User Input: The program waits for user input via `input("You: ")`.

- Exit Condition: If the user types `"exit"` or `"quit"`, the loop breaks and the chat ends.

- Agent Response: The `chat_with_agent` function is called with the user’s input and chat history. The returned `history` includes both the user’s and AI’s messages.

- Display User Message and AI Reply

**Purpose**

This loop turns your agent into an interactive chatbot that remembers the last few messages, responds in real time, and ends when the user types *exit* or *quit*.


In [31]:
history = []
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    history, new_messages = chat_with_agent(user_input, history)
    print(f"You: {user_input}") # Display the user's input
    
    # Iterate and display all new messages generated by the graph run
    for msg in new_messages:
        # Determine the prefix based on message type
        prefix = "AI:"
        if isinstance(msg, HumanMessage):
            # This should rarely happen but is a good safeguard
            prefix = "Human:" 
        
        # Print the message content
        print(f"{prefix} {msg.content}")
    
    # Add a separator for the next turn
    print("---------------------------------------------\n")

You: how do i connect my wallet
AI: To connect your wallet, you'll typically need to:

1.  **Have a compatible wallet:** Make sure you have a wallet like MetaMask, Coinbase Wallet, or another Base-compatible wallet installed.
2.  **Visit a dApp:** Go to a decentralized application (dApp) on Base that requires wallet connection.
3.  **Click "Connect Wallet":** Look for a button that says "Connect Wallet" or similar, usually in the top right corner of the dApp.
4.  **Select your wallet:** Choose your wallet provider from the options.
5.  **Approve connection:** Your wallet will pop up asking for permission to connect to the dApp. Approve the request.

Once connected, the dApp will be able to interact with your wallet for transactions.
---------------------------------------------

You: i want to connect my wallet, can you guide me 
AI: I can guide you through the general steps of connecting your wallet to a dApp on Base. Could you tell me which wallet you are using (e.g., MetaMask, Coinb